### IMPORTS

In [1]:
import torch
from torch import nn
from torch import functional as F
import matplotlib.pyplot as plt
import numpy as np
from livelossplot import PlotLosses
from Tools.NNtools import *
import math

In [2]:
# Find what device to work with
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [3]:
def identity(m):
    return m

In [4]:
class parallel_MLP(nn.Module):
            def __init__(self, layerwidth,nb_layers=1):
                super(parallel_MLP, self).__init__()
                self.layerwidth = layerwidth
                self.activation=nn.Tanh()# use identity for debugging
                self.requires_grad_(False)
                self.nb_layers=nb_layers
                self.param_count=2*layerwidth+(nb_layers-1)*(layerwidth**2+layerwidth)+layerwidth+1
                self.splitting=[layerwidth]*2+[layerwidth**2,layerwidth]*(nb_layers-1)+[layerwidth,1]
                
            def forward(self,theta,x):
                nb_theta=theta.shape[0]
                nb_x=x.shape[0]
                theta=theta.split(self.splitting,dim=1)
                input_x=x.view(nb_x,1,1)
                print(input_x.shape)
                m=torch.matmul(theta[0].view(nb_theta,1,self.layerwidth,1),input_x)
                print(m.shape)
                m=m.add(theta[1].reshape(nb_theta,1,self.layerwidth,1))
                print(m.shape)
                m=self.activation(m)
                print(m.shape)
                for i in range(self.nb_layers-1):
                    print(m.shape)                   
                    m=torch.matmul(theta[2*i+2].view(-1,1,self.layerwidth,self.layerwidth),m)
                    print(m.shape)
                    m=m.add(theta[2*i+3].reshape(-1,1,self.layerwidth,1))
                    print(m.shape)
                    m=self.activation(m)
                    print(m.shape)
                m=torch.matmul(theta[2*(self.nb_layers-1)+2].view(nb_theta,1,1,self.layerwidth),m)
                print(m.shape)
                m=m.add(theta[2*(self.nb_layers-1)+3].reshape(nb_theta,1,1,1))
                print(m.shape)
                return m.squeeze(-1)


In [5]:
model=parallel_MLP(100,4)

In [6]:

model.splitting

[100, 100, 10000, 100, 10000, 100, 10000, 100, 100, 1]

In [7]:
param_count=model.param_count
param_count

30601

In [8]:
x=torch.rand(1,1)

theta=torch.rand(1,param_count)

model(theta,x)

torch.Size([1, 1, 1])
torch.Size([1, 1, 100, 1])
torch.Size([1, 1, 100, 1])
torch.Size([1, 1, 100, 1])
torch.Size([1, 1, 100, 1])
torch.Size([1, 1, 100, 1])
torch.Size([1, 1, 100, 1])
torch.Size([1, 1, 100, 1])
torch.Size([1, 1, 100, 1])
torch.Size([1, 1, 100, 1])
torch.Size([1, 1, 100, 1])
torch.Size([1, 1, 100, 1])
torch.Size([1, 1, 100, 1])
torch.Size([1, 1, 100, 1])
torch.Size([1, 1, 100, 1])
torch.Size([1, 1, 100, 1])
torch.Size([1, 1, 1, 1])
torch.Size([1, 1, 1, 1])


tensor([[[58.5885]]])

In [9]:
x=torch.rand(1,1)

theta=torch.rand(1,param_count)
theta_=torch.rand(1,param_count)

y=model(theta,x)
y_=model(theta_,x)
y_m=torch.cat([y,y_],dim=0)
y_M=model(torch.stack([theta.squeeze(),theta_.squeeze()]),x)
#print(torch.stack([y_m,y_M],dim=2))
torch.eq(y_m,y_M)
#y_m,y_M

torch.Size([1, 1, 1])
torch.Size([1, 1, 100, 1])
torch.Size([1, 1, 100, 1])
torch.Size([1, 1, 100, 1])
torch.Size([1, 1, 100, 1])
torch.Size([1, 1, 100, 1])
torch.Size([1, 1, 100, 1])
torch.Size([1, 1, 100, 1])
torch.Size([1, 1, 100, 1])
torch.Size([1, 1, 100, 1])
torch.Size([1, 1, 100, 1])
torch.Size([1, 1, 100, 1])
torch.Size([1, 1, 100, 1])
torch.Size([1, 1, 100, 1])
torch.Size([1, 1, 100, 1])
torch.Size([1, 1, 100, 1])
torch.Size([1, 1, 1, 1])
torch.Size([1, 1, 1, 1])
torch.Size([1, 1, 1])
torch.Size([1, 1, 100, 1])
torch.Size([1, 1, 100, 1])
torch.Size([1, 1, 100, 1])
torch.Size([1, 1, 100, 1])
torch.Size([1, 1, 100, 1])
torch.Size([1, 1, 100, 1])
torch.Size([1, 1, 100, 1])
torch.Size([1, 1, 100, 1])
torch.Size([1, 1, 100, 1])
torch.Size([1, 1, 100, 1])
torch.Size([1, 1, 100, 1])
torch.Size([1, 1, 100, 1])
torch.Size([1, 1, 100, 1])
torch.Size([1, 1, 100, 1])
torch.Size([1, 1, 100, 1])
torch.Size([1, 1, 1, 1])
torch.Size([1, 1, 1, 1])
torch.Size([1, 1, 1])
torch.Size([2, 1, 100, 1

tensor([[[True]],

        [[True]]])

In [10]:
x=torch.rand(3,1)
x_=torch.rand(5,1)
theta=torch.rand(1,param_count)

y=model(theta,x)
y_=model(theta,x_)
y_m=torch.cat([y,y_],dim=1)
y_M=model(theta,torch.cat([x,x_],dim=0))
#print(torch.stack([y_m,y_M],dim=2))
print(y_m==y_M)

torch.Size([3, 1, 1])
torch.Size([1, 3, 100, 1])
torch.Size([1, 3, 100, 1])
torch.Size([1, 3, 100, 1])
torch.Size([1, 3, 100, 1])
torch.Size([1, 3, 100, 1])
torch.Size([1, 3, 100, 1])
torch.Size([1, 3, 100, 1])
torch.Size([1, 3, 100, 1])
torch.Size([1, 3, 100, 1])
torch.Size([1, 3, 100, 1])
torch.Size([1, 3, 100, 1])
torch.Size([1, 3, 100, 1])
torch.Size([1, 3, 100, 1])
torch.Size([1, 3, 100, 1])
torch.Size([1, 3, 100, 1])
torch.Size([1, 3, 1, 1])
torch.Size([1, 3, 1, 1])
torch.Size([5, 1, 1])
torch.Size([1, 5, 100, 1])
torch.Size([1, 5, 100, 1])
torch.Size([1, 5, 100, 1])
torch.Size([1, 5, 100, 1])
torch.Size([1, 5, 100, 1])
torch.Size([1, 5, 100, 1])
torch.Size([1, 5, 100, 1])
torch.Size([1, 5, 100, 1])
torch.Size([1, 5, 100, 1])
torch.Size([1, 5, 100, 1])
torch.Size([1, 5, 100, 1])
torch.Size([1, 5, 100, 1])
torch.Size([1, 5, 100, 1])
torch.Size([1, 5, 100, 1])
torch.Size([1, 5, 100, 1])
torch.Size([1, 5, 1, 1])
torch.Size([1, 5, 1, 1])
torch.Size([8, 1, 1])
torch.Size([1, 8, 100, 1

In [14]:
x=torch.rand(200,1)
theta=torch.rand(100,param_count)

y=model(theta,x)

torch.Size([200, 1, 1])
torch.Size([100, 200, 100, 1])
torch.Size([100, 200, 100, 1])
torch.Size([100, 200, 100, 1])
torch.Size([100, 200, 100, 1])
torch.Size([100, 200, 100, 1])
torch.Size([100, 200, 100, 1])
torch.Size([100, 200, 100, 1])
torch.Size([100, 200, 100, 1])
torch.Size([100, 200, 100, 1])
torch.Size([100, 200, 100, 1])
torch.Size([100, 200, 100, 1])
torch.Size([100, 200, 100, 1])
torch.Size([100, 200, 100, 1])
torch.Size([100, 200, 100, 1])
torch.Size([100, 200, 100, 1])
torch.Size([100, 200, 1, 1])
torch.Size([100, 200, 1, 1])
